In [2]:
import numpy as np
import pandas as pd

In [3]:
ease = pd.read_csv('./output/1_EASE_epoch10.csv')
bert = pd.read_csv('./output/2_EASE_newversion.csv')

In [4]:
ease8 = ease.groupby('user').head(8)
ease8.head(10)

,user,item
0,11,4886
1,11,4370
2,11,8961
3,11,40815
4,11,2329
5,11,32587
6,11,37386
7,11,47
10,14,1198
11,14,2011


In [5]:
bert['prior'] = [i for i in range(8,18)]*31360

In [6]:
result = pd.concat([ease8, bert])
result

,user,item,prior
0,11,4886,NaN
1,11,4370,NaN
2,11,8961,NaN
3,11,40815,NaN
4,11,2329,NaN
...,...,...,...
313595,138493,593,13.0
313596,138493,4848,14.0
313597,138493,4022,15.0
313598,138493,53125,16.0


In [7]:
result = result.drop_duplicates(['user','item'],keep='first')
result

,user,item,prior
0,11,4886,NaN
1,11,4370,NaN
2,11,8961,NaN
3,11,40815,NaN
4,11,2329,NaN
...,...,...,...
313589,138492,1210,17.0
313592,138493,293,10.0
313596,138493,4848,14.0
313597,138493,4022,15.0


In [8]:
result = result.sort_values(['user','prior'])

In [9]:
result = result.groupby('user').head(10)
result

,user,item,prior
4,11,1704,12.0
6,11,3996,14.0
8,11,1127,16.0
0,11,4886,NaN
1,11,4370,NaN
...,...,...,...
313591,138493,2628,NaN
313592,138493,593,NaN
313593,138493,110,NaN
313594,138493,1270,NaN


In [ ]:
result[['user','item']].to_csv('./output/ratio_ensemble.csv',index=False)